In [1]:
import pandas as pd
from datasets import Dataset

data = pd.read_csv("datasets/dataset_v1.csv")
tuning_examples = []

for index, row in data.iterrows():
    tuning_examples.append(f"<|user|>\n{row['instruction']}:\n{row['input']}\n<|assistant|>\n{row['output']}\n<|endoftext|>")

dataset = Dataset.from_dict({"text": tuning_examples})


/home/gustavo/Git/AI_Fine_Tuning/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from unsloth import FastLanguageModel

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "meta-llama/Llama-3.2-3B-Instruct",
    max_seq_length = 2048,
    dtype = None,
    load_in_4bit = True
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.12.8: Fast Llama patching. Transformers: 4.57.3.
   \\   /|    NVIDIA GeForce RTX 4060. Num GPUs = 1. Max memory: 7.617 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.1+cu128. CUDA: 8.9. CUDA Toolkit: 12.8. Triton: 3.5.1
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.33.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [3]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 64,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_alpha = 64 * 2,
    lora_dropout = 0,
    bias = 'none',
    use_gradient_checkpointing = 'unsloth'
)

Unsloth 2025.12.8 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [4]:
from trl import SFTTrainer, SFTConfig

trainer = SFTTrainer(
    model = model,
    train_dataset = dataset,
    tokenizer = tokenizer,
    dataset_text = 'text',
    max_seq_length = 2048,
    args = SFTConfig(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 10,
        max_steps = 60,
        num_train_epochs = 3,
        logging_steps = 1,
        output_dir = 'outputs',
        optim = 'adamw_8bit'
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=16): 100%|██████████| 17/17 [00:03<00:00,  4.56 examples/s]


🦥 Unsloth: Padding-free auto-enabled, enabling faster training.


In [5]:
trainer.train()

The model is already on multiple devices. Skipping the move to device specified in `args`.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 17 | Num Epochs = 20 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 97,255,424 of 3,310,005,248 (2.94% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,3.037200
2,2.454700
3,2.662700
4,2.466700
5,2.650300
6,3.076700
7,2.534600
8,1.986200
9,1.709900
10,1.645000


TrainOutput(global_step=60, training_loss=0.7255418534701069, metrics={'train_runtime': 76.2842, 'train_samples_per_second': 6.292, 'train_steps_per_second': 0.787, 'total_flos': 824412316139520.0, 'train_loss': 0.7255418534701069, 'epoch': 20.0})

In [6]:
FastLanguageModel.for_inference(model)

messages = [
    {
        "role": "user",
        "content": "Converta esse script do IBM RPA para Python: \n\ndefVar --name fruit --type String --value \"Melancia Comprida\""
    }
]

inputs = tokenizer.apply_chat_template(messages, tokenize=True, add_generation_prompt=True, return_tensors="pt").to("cuda")
outputs = model.generate(input_ids=inputs, max_new_tokens=512, use_cache=True, temperature=0.7, do_sample=True, top_p=0.9)
response = tokenizer.batch_decode(outputs)[0]

print(response)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 29 Dec 2025

<|eot_id|><|start_header_id|>user<|end_header_id|>

Converta esse script do IBM RPA para Python: 

defVar --name fruit --type String --value "Melancia Comprida"<|eot_id|><|start_header_id|>assistant<|end_header_id|>

fruit = "Melancia Comprida"
print(fruit)<|eot_id|>


In [7]:
model.save_pretrained_gguf("finetuned_model", tokenizer, quantization_method="q4_k_m", maximum_memory_usage=0.3)

Unsloth: Merging model weights to 16-bit format...
Found HuggingFace hub cache directory: /home/gustavo/.cache/huggingface/hub


Fetching 1 files: 100%|██████████| 1/1 [00:00<00:00,  1.67it/s]


Checking cache directory for required files...
Cache check failed: model-00001-of-00002.safetensors not found in local cache.
Not all required files found in cache. Will proceed with downloading.
Checking cache directory for required files...
Cache check failed: tokenizer.model not found in local cache.
Not all required files found in cache. Will proceed with downloading.


Unsloth: Preparing safetensor model files: 100%|██████████| 2/2 [01:50<00:00, 55.36s/it]


Note: tokenizer.model not found (this is OK for non-SentencePiece models)


Unsloth: Merging weights into 16bit: 100%|██████████| 2/2 [00:12<00:00,  6.31s/it]


Unsloth: Merge process complete. Saved to `/home/gustavo/Git/AI_Fine_Tuning/finetuned_model`
Unsloth: Converting to GGUF format...
==((====))==  Unsloth: Conversion from HF to GGUF information
   \\   /|    [0] Installing llama.cpp might take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GGUF bf16 might take 3 minutes.
\        /    [2] Converting GGUF bf16 to ['q4_k_m'] might take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: llama.cpp found in the system. Skipping installation.
Unsloth: Preparing converter script...
Unsloth: [1] Converting model into bf16 GGUF format.
This might take 3 minutes...
Unsloth: Initial conversion completed! Files: ['llama-3.2-3b-instruct.BF16.gguf']
Unsloth: [2] Converting GGUF bf16 into q4_k_m. This might take 10 minutes...
Unsloth: Model files cleanup...
Unsloth: All GGUF conversions completed successfully!
Generated files: ['llama-3.2-3b-instruct.Q4_K_M.gguf']
Unsloth: example usage for text only LLMs: lla

{'save_directory': 'finetuned_model',
 'gguf_files': ['llama-3.2-3b-instruct.Q4_K_M.gguf'],
 'modelfile_location': '/home/gustavo/Git/AI_Fine_Tuning/Modelfile',
 'want_full_precision': False,
 'is_vlm': False,
 'fix_bos_token': False}